In [1]:
# Importing packages
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import IFrame
import matplotlib.patheffects as PathEffects

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
trailname = 'gr131' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)
buffersize = 0.00015 # how many degs to buffer by
delta_places = 0.015 # bbox delta in deg

# n1 = 2300
# n2 = 2400
n1 = 4000
n2 = 4150

In [3]:
trail = gr_utils.get_gpx(trailname)
data_roads = gr_utils.read_roads(trailname)
bbox = gr_placematch.get_bbox(data_roads[n1:n2], delta_places);
places_landuse, places_admin8, places_admin9 = gr_placematch.get_places(bbox) # Grab relevant place information
i_focus = int(np.round((n1+n2)/2))
focus = [data_roads.iloc[i_focus]['x0'],data_roads.iloc[i_focus]['y0']]

Loading trail points from <data_input/gr131.gpx>...
Finished loading.


## Drawing large merged polygons

In [4]:
polys = []
for i in range(places_landuse.shape[0]):
    poly = places_landuse.iloc[i]['geometry'] # Grab polygon
#     poly = shapely.affinity.scale(poly, xfact=sc, yfact=sc, origin='center') # Scale it
    poly = poly.buffer(buffersize)
    polys.append(poly)
merged = list(shapely.ops.unary_union(polys))

/var/folders/zh/x06gqp657rl9h1shq_pn7gjh0000gn/T/ipykernel_8277/1521081679.py:7: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged = list(shapely.ops.unary_union(polys))
/var/folders/zh/x06gqp657rl9h1shq_pn7gjh0000gn/T/ipykernel_8277/1521081679.py:7: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged = list(shapely.ops.unary_union(polys))


In [5]:
tol_area = 15.0e-6

# Map setup
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
k = 0
for poly in merged:

    x,y = poly.exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    if poly.area>tol_area:
        newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, popup=poly.area, weight=2)
    else:
        newline = folium.Polygon(locations=coords, color='green', fill=True, fillOpacity=0.1, popup=poly.area, weight=1)
    newline.add_to(chart)
    k += 1

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
newline = folium.PolyLine(locations=coords[n1:n2], weight=2, color='red')
newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_large_devs.html"
chart.save(filepath)

In [6]:
IFrame(filepath, width=1000, height=500)

## Draw where trail passes near large developed areas

In [9]:
tol_area = 15.0e-6
tol_near = 5e-4

# Map setup
chart = folium.Map(location=focus, zoom_start=13, tiles="OpenStreetMap")

# Draw development areas
k = 0
for poly in merged:

    x,y = poly.exterior.coords.xy
    coords = pd.DataFrame({'LAT':y,'LON':x})
    if poly.area>tol_area:
        newline = folium.Polygon(locations=coords, color='blue', fill=True, fillOpacity=0.5, popup=poly.area, weight=2)
    else:
        newline = folium.Polygon(locations=coords, color='green', fill=True, fillOpacity=0.1, popup=poly.area, weight=1)
    newline.add_to(chart)
    k += 1

# Draw trail
coords = gr_plot.get_fullcoords_from_frame(data_roads)
for i in range(n1,n2):
    xmid = (coords[i][0] + coords[i+1][0])/2
    ymid = (coords[i][1] + coords[i+1][1])/2
    mid = shapely.geometry.point.Point([ymid,xmid])
    d = 999.9;
    for poly in merged:
        if poly.area>tol_area:
            new_d = poly.distance(mid) # Calculate distance midpoint - polygon
            d = min(d, new_d)
    if d<tol_near:
        color = 'red'
    else:
        color = 'black'
#     print(d)
#     color = 'black'
    newline = folium.PolyLine(locations=coords[i:i+2], weight=2, color=color)
    newline.add_to(chart)

# Draw bounding box
x,y = bbox.exterior.coords.xy
coords = pd.DataFrame({'LAT':y,'LON':x})
newline = folium.Polygon(locations=coords, color='black', fill=False, weight=2)
newline.add_to(chart)

# Render the map
filepath = f"cache/plot_large_devs_trail.html"
chart.save(filepath)

In [10]:
IFrame(filepath, width=1000, height=500)